In [6]:
'''
async def run_tasks(handles, interval, output_path):
    with open(output_path, 'wb') as output:
        async def write_async(data):
            output.write(data)
        
        tasks = []
        
        for handle in handles:
            coro = tail_async(handle, interval, write_async)
            task = asyncio.create_task(coro)
            tasks.append(task)
            
        await asyncio.gather(*tasks)
        
# 이 코드에도 한가지 문제가 있다. 출력 파일 핸들에 대한 open, close, write 호출이 주 이벤트 루프에서 이뤄져야 한다는 것이다.
# 이런 연산은 모두 프로그램을 실행하는 운영체제의 시스템 콜을 사용하기 때문에 이벤트 루프를 상당히 오랫동안 블록할 수 있으므로 다른 코루틴이 진행하지 못하게 된다.

# debug=True 파라미터를 asyncio.run 함수에 넘기면 된다
'''

import time
import asyncio

async def slow_corutine():
    time.sleep(0.5) # 느린 I/O 시뮬레이션
    
#asyncio.run(slow_corutine(), debug=True)
await slow_corutine() # use jupyter
    



        




TypeError: slow_corutine() got an unexpected keyword argument 'debug'

In [ ]:
from threading import Thread

class WriteThread(Thread):
    def __init__(self, output_path):
        super().__init__()
        self.output_path = output_path
        self.output = None
        self.loop = asyncio.new_event_loop()
    
    def run(self):
        asyncio.set_event_loop(self.loop)
        with open(self.output_path, 'wb') as self.output:
            self.loop.run_forever()
            
        # 맨 마지막에 한 번 더 이벤트 루프를 실행해서
        # 다른 이벤트 루프가 stop()에 await하는 경우를 해결한다
        self.loop.run_until_complete(asyncio.sleep(0))
        
        
    async def real_write(self, data):
        self.output.write(data)
        
    async def write(self, data):
        coro = self.real_write(data) # 실제 I/O를 실행하는 메서드를 스레드 안전하게 감싸준다
        future = asyncio.run_coroutine_threadsafe(
            coro, self.loop
        )
        await asyncio.wrap_future(future)
        
    async def real_stop(self):
        self.loop.stop()
        
    async def stop(self):
        coro = self.real_stop()
        future = asyncio.run_coroutine_threadsafe(
            coro, self.loop
        )
        await asyncio.wrap_future(future)
        
    async def __aenter__(self):
        loop = asyncio.get_event_loop()
        await loop.run_in_executor(None, self.start)
        return self

    async def __aexit__(self, *_):
        await self.stop()